# 1.数据集的构建

如果使用我的数据集 ：

images https://drive.google.com/drive/folders/1y8vJjChXGWeSn3wq-VGPZlfLN3z2Pxsy?usp=sharing

Annotations：https://drive.google.com/drive/folders/1UoGZ4f8ac9nOoW7JUEIoySf9DZb0ggBJ?usp=sharing



将其相关文件夹保存在data目录下后进行训练，我已经保存了所有的训练集及数据集，如果是使用自己的数据集请按说明保存。

整个文件中，在data路径下新建images，ImagesSets， Annotations和labels文件夹，images文件夹下放入图像文件，Annotations文件夹下放入xml文件，运行下列程序生成训练集和测试集：

In [ ]:

import os
import random
 
trainval_percent = 0.1
train_percent = 0.9
xmlfilepath = 'data/Annotations'
txtsavepath = 'data/images'
total_xml = os.listdir(xmlfilepath)
 
num = len(total_xml)
list = range(num)
tv = int(num * trainval_percent)
tr = int(tv * train_percent)
trainval = random.sample(list, tv)
train = random.sample(trainval, tr)
 
ftrainval = open('data/ImageSets/trainval.txt', 'w')
ftest = open('data/ImageSets/test.txt', 'w')
ftrain = open('data/ImageSets/train.txt', 'w')
fval = open('data/ImageSets/val.txt', 'w')

a=0
for i in list:
    a+=1
    name = total_xml[i][:-4] + '\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftest.write(name)
        else:
            fval.write(name)
    else:
        ftrain.write(name)
        
# print(a)
ftrainval.close()
ftrain.close()
fval.close()
ftest.close()

生成txt文件的labels并将其保存在labels文件夹中，并在yolov3根目录下生成训练集路径及测试集路径, 由于我最开始已经保存了训练集及测试集，如果需要训练自己的数据集请先删除我先前保存的train.txt以及test.txt文y件

In [ ]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join
 
sets = ['train', 'test','val']
 
# since we only have one class, we can use list
classes = ["light"] 
 
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(image_id):
    in_file = open('data/Annotations/%s.xml' % (image_id))
    out_file = open('data/labels/%s.txt' % (image_id), 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
 
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text),
             float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
 
 
wd = getcwd()
print(wd)
for image_set in sets:
    if not os.path.exists('data/labels/'):
        os.makedirs('data/labels/')
    image_ids = open('data/ImageSets/%s.txt' % (image_set)).read().strip().split()
    list_file = open('data/%s.txt' % (image_set), 'w')
    for image_id in image_ids:
        list_file.write('data/images/%s.jpg\n' % (image_id))
        convert_annotation(image_id)
    list_file.close()

# 2.模型的训练

由于本人默认是使用yolov3-tiny进行训练，class的数量默认为1，图像大小默认为128*128，如果训练图像尺寸及相关数据不同的话请注意修改。


训练结果保存在weight文件夹下。


使用的weight为从darknet进行train得到的yolov3-tiny.conv.15

In [ ]:
#this model can use both cpu and gpu, but highly recommed gpu enviroment, since it can be much much faster!
#check device, if device is GPU the result would be cuda
import thorch
import numpy as np
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [ ]:
!python3 train.py --epochs 300 --weights weights/last.pt --batch-size 16 --img-size 128

# 3.结果测试

将需要测试的图片保存在samples文件夹中，输出的图像结果以及txt文件将保存在根目录的output文件夹下。


txt格式为：class xmin xmax ymin ymax

In [ ]:
!python detect1.py --data data/coco.data --cfg cfg/yolov3-tiny.cfg --weights weights/best.pt  --img-size 128

可以利用Image 函数查看output里的图片结果

In [ ]:
from IPython.display import Image, clear_output 
Image(filename='output/821.jpg', width=300)